In [125]:
##Optional, could also use num2str or something else. 
!pip install inflect 

Defaulting to user installation because normal site-packages is not writeable


In [160]:
#Import Required Libraries and Packages
import numpy as np
import pandas as pd 
import inflect as inf
import re
from re import sub
import requests
import bs4

In [161]:
##########################################...HELPER FUNCTIONS...####################################################

In [162]:
###CAMELCASE FUNCTION###
def camel_case(s):
    #not true camelcase but adjusted for queries 
    
  s = sub(r"(_|-)+", " ", s).title().replace(" ", "")
  return ''.join([s[0].upper(), s[1:]])


In [170]:
oscars = pd.read_csv("BigML_Dataset_5f50a62c2fb31c516d000176.csv")
oscars['genre'] = [str(i).split("|") for i in oscars["genre"]]
oscars

,year,movie,movie_id,certificate,duration,genre,rate,metascore,synopsis,votes,...,New_York_Film_Critics_Circle_nominated,New_York_Film_Critics_Circle_nominated_categories,Los_Angeles_Film_Critics_Association_won,Los_Angeles_Film_Critics_Association_won_categories,Los_Angeles_Film_Critics_Association_nominated,Los_Angeles_Film_Critics_Association_nominated_categories,release_date.year,release_date.month,release_date.day-of-month,release_date.day-of-week
0,2001,Kate & Leopold,tt0035423,PG-13,118,"[Comedy, Fantasy, Romance]",6.4,44.0,An English Duke from 1876 is inadvertedly drag...,66660,...,0,NaN,0,NaN,0,NaN,2001.0,12.0,25.0,2.0
1,2000,Chicken Run,tt0120630,G,84,"[Animation, Adventure, Comedy]",7.0,88.0,When a cockerel apparently flies into a chicke...,144475,...,1,Best Animated Film,1,Best Animation,1,Best Animation,2000.0,6.0,23.0,5.0
2,2005,Fantastic Four,tt0120667,PG-13,106,"[Action, Adventure, Family]",5.7,40.0,A group of astronauts gain superpowers after a...,273203,...,0,NaN,0,NaN,0,NaN,2005.0,7.0,8.0,5.0
3,2002,Frida,tt0120679,R,123,"[Biography, Drama, Romance]",7.4,61.0,"A biography of artist Frida Kahlo, who channel...",63852,...,0,NaN,0,NaN,0,NaN,2002.0,11.0,22.0,5.0
4,2001,The Lord of the Rings: The Fellowship of the Ring,tt0120737,PG-13,178,"[Adventure, Drama, Fantasy]",8.8,92.0,A meek Hobbit from the Shire and eight compani...,1286275,...,0,NaN,1,Best Music,2,Best Music|Best Production Design,2001.0,12.0,19.0,3.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1178,2017,Call Me by Your Name,tt5726616,R,132,"[Drama, Romance]",8.3,93.0,"In Northern Italy in 1983, seventeen year-old ...",38170,...,1,Best Actor,3,Best Actor|Best Director|Best Picture,3,Best Actor|Best Director|Best Picture,NaN,NaN,NaN,NaN
1179,2017,Phantom Thread,tt5776858,R,130,"[Drama, Romance]",8.4,90.0,"Set in 1950's London, Reynolds Woodcock is a r...",7380,...,1,Best Screenplay,1,Best Music,1,Best Music,NaN,NaN,NaN,NaN
1180,2017,Victoria & Abdul,tt5816682,PG-13,111,"[Biography, Drama, History]",6.8,58.0,Queen Victoria strikes up an unlikely friendsh...,12888,...,0,NaN,0,NaN,0,NaN,2017.0,10.0,6.0,5.0
1181,2017,"Roman J. Israel, Esq.",tt6000478,PG-13,122,"[Crime, Drama]",6.3,58.0,"Roman J. Israel, Esq., a driven, idealistic de...",3205,...,0,NaN,0,NaN,0,NaN,NaN,NaN,NaN,NaN


In [164]:
##OSCAR DATA, we need to keep title, some numeric ratings, year, genre, and whether or not it was simply nominated 
# or it won

##Then We need a way to scrape movie tropes and keep them organized in a list 

In [197]:
oscar = oscars[["movie", 'year','release_date.year', 'Oscar_nominated', 'Oscar_nominated_categories', 'Oscar_Best_Picture_won' ,'gross' ,'genre','rate','metascore']].copy()
oscar.sort_values(['year', 'movie'])

#if there's numbers in the title then they must be convertyed into text. We can write a small function to do that
# or use a library like num2str or inflect
p = inf.engine()





#find those titles with numbers in their names and correct them 

titles = [oscar["movie"][i] for i in range(0, len(list(oscar["movie"])))]
#Read the title
i = 0
for original in titles:
    
    t = ''.join(re.findall(r'[0-9]+', original))
    
    if (t == '' or original =='Thir13en Ghosts' or original == 'U-571'):
        pass
    else:
        nw = p.number_to_words(original)
        titles[i] = original.replace(t,nw)  
        
    i+=1

oscar.loc[:,'movie'] = titles
oscar.head(50)








#re.subn(r"[0-9]", p.number_to_words(), original)

#two problems here--one; num_to_words, includes - back. so this needs to occur on oscar itself--SOLVED
#second, this doesnt handle multiple numbers, think for a little bit




,movie,year,release_date.year,Oscar_nominated,Oscar_nominated_categories,Oscar_Best_Picture_won,gross,genre,rate,metascore
0,Kate & Leopold,2001,2001.0,1,"Best Music, Original Song",No,47100000.0,"[Comedy, Fantasy, Romance]",6.4,44.0
1,Chicken Run,2000,2000.0,0,NaN,No,106790000.0,"[Animation, Adventure, Comedy]",7.0,88.0
2,Fantastic Four,2005,2005.0,0,NaN,No,154700000.0,"[Action, Adventure, Family]",5.7,40.0
3,Frida,2002,2002.0,6,"Best Music, Original Score|Best Makeup|Best Pe...",No,25780000.0,"[Biography, Drama, Romance]",7.4,61.0
4,The Lord of the Rings: The Fellowship of the Ring,2001,2001.0,13,"Best Cinematography|Best Makeup|Best Music, Or...",No,313840000.0,"[Adventure, Drama, Fantasy]",8.8,92.0
5,Mission: Impossible II,2000,2000.0,0,NaN,No,215400000.0,"[Action, Adventure, Thriller]",6.1,59.0
6,Resident Evil,2002,2002.0,0,NaN,No,39530000.0,"[Action, Horror, Sci-Fi]",6.7,33.0
7,X-Men,2000,2000.0,0,NaN,No,157300000.0,"[Action, Adventure, Sci-Fi]",7.4,64.0
8,The Emperor's New Groove,2000,2000.0,1,"Best Music, Original Song",No,89300000.0,"[Animation, Adventure, Comedy]",7.3,70.0
9,Corpse Bride,2005,2005.0,1,Best Animated Feature Film of the Year,No,53340000.0,"[Animation, Drama, Family]",7.4,83.0


In [199]:


##Some more Data cleanup, remove the :, --, &-->and

and_oscar = oscar[["&" in titl for titl in oscar["movie"]]]
oscar["movie"].loc[["&" in titl for titl in oscar["movie"]] ] = [ti.replace("&", "and") for ti in and_oscar["movie"] ]

col_oscar = oscar [[":" in titl for titl in oscar["movie"]]]
oscar["movie"].loc[[":" in titl for titl in oscar["movie"]]] = [ti.replace(":", "") for ti in col_oscar["movie"] ]

#dd_oscar =
dd_oscar = oscar [["--" in titl for titl in oscar["movie"]]]
oscar["movie"].loc[["--" in titl for titl in oscar["movie"]]] = [ti.replace("--", "") for ti in dd_oscar["movie"] ]


#dash_oscar =
dash_oscar = oscar [["-" in titl for titl in oscar["movie"]]]
oscar["movie"].loc[["-" in titl for titl in oscar["movie"]]] = [ti.replace("-", "") for ti in dash_oscar["movie"] ]

#com_oscar =
com_oscar = oscar [["," in titl for titl in oscar["movie"]]]
oscar["movie"].loc[["," in titl for titl in oscar["movie"]]] = [ti.replace(",", "") for ti in com_oscar["movie"] ]

#pd_oscar =
pd_oscar = oscar [["." in titl for titl in oscar["movie"]]]
oscar["movie"].loc[["." in titl for titl in oscar["movie"]]] = [ti.replace(".", "") for ti in pd_oscar["movie"] ]

#ap_oscar
ap_oscar = oscar [["''" in titl for titl in oscar["movie"]]]
oscar["movie"].loc[["''" in titl for titl in oscar["movie"]]] = [ti.replace("''", "") for ti in ap_oscar["movie"] ]



#oscar["movie"].loc[[("-" or "--" or ":" or ',') in titl for titl in oscar["movie"]]] =  [ti.replace("-", "").replace('--','').replace(',','').replace(":", "") for ti in oscar["movies"][] ]
#andtitles = list(filter(lambda ti: "&" in ti ,titles))
#[ ti.replace("&", "and")  for ti in andtitles ]

#All Title Cleaning Done, 
oscar


,movie,year,release_date.year,Oscar_nominated,Oscar_nominated_categories,Oscar_Best_Picture_won,gross,genre,rate,metascore
0,Kate and Leopold,2001,2001.0,1,"Best Music, Original Song",No,47100000.0,"[Comedy, Fantasy, Romance]",6.4,44.0
1,Chicken Run,2000,2000.0,0,NaN,No,106790000.0,"[Animation, Adventure, Comedy]",7.0,88.0
2,Fantastic Four,2005,2005.0,0,NaN,No,154700000.0,"[Action, Adventure, Family]",5.7,40.0
3,Frida,2002,2002.0,6,"Best Music, Original Score|Best Makeup|Best Pe...",No,25780000.0,"[Biography, Drama, Romance]",7.4,61.0
4,The Lord of the Rings The Fellowship of the Ring,2001,2001.0,13,"Best Cinematography|Best Makeup|Best Music, Or...",No,313840000.0,"[Adventure, Drama, Fantasy]",8.8,92.0
...,...,...,...,...,...,...,...,...,...,...
1178,Call Me by Your Name,2017,NaN,4,Best Performance by an Actor in a Leading Role...,No,11300000.0,"[Drama, Romance]",8.3,93.0
1179,Phantom Thread,2017,NaN,6,Best Performance by an Actor in a Leading Role...,No,10710000.0,"[Drama, Romance]",8.4,90.0
1180,Victoria and Abdul,2017,2017.0,2,Best Achievement in Makeup and Hairstyling|Bes...,No,22250000.0,"[Biography, Drama, History]",6.8,58.0
1181,Roman J Israel Esq,2017,NaN,1,Best Performance by an Actor in a Leading Role,No,11940000.0,"[Crime, Drama]",6.3,58.0


In [ ]:
##Scrape Data from TV tropes--
# /Film for the film. 
# Place Animated movies in different places

#then, we read the web, and scrape the data that we need from the website
#this will be the t

trope_dict= {}


for i in titles:
            ## Ex. Star Wars : Ep II -- Arr.... only the Attack of the Clones Part is important.
            # & must be converted too and other delimeters must be cleaned
    try:
            filmtitle = camel_case(titles[3])
            anititle = camel_case(titles[1])
            print(filmtitle)
            urlFilm = f'https://tvtropes.org/pmwiki/pmwiki.php/Film/{filmtitle}'
            urlAnim = f'https://tvtropes.org/pmwiki/pmwiki.php/WesternAnimation/{anititle}'


            #GET PAGE 
            page = requests.get(urlFilm)



            #BS the rquest 
            soup = bs4.BeautifulSoup(page.content, 'html.parser')

            #Get all twikilink's
            anchors = soup.find_all('a', attrs = {'class' : 'twikilink',  })
            
            
            #Get all the hrefs from them, then get only those that head to the Main trope page
            #Make a set of all the tropes found in the article
            links = [link.get('href') for link in anchors]
            mainlinks = filter(lambda l: '/pmwiki/pmwiki.php/Main/' in l , links)
            tropes = [i.split('/pmwiki/pmwiki.php/Main/')[-1] for i in mainlinks]
            tset = set(tropes)
            print(tset)
            
            trope_dict[i] = tset

            #Make dict with key as name and tset as entry
            


    except:
        pass

trope_dict

#3,6. 




